# Token classification
This notebook shows our approach to the data preprocessing. The goal is to have exactly one label per token*, including an "empty" label.
As long as we restrict our data to only one predicate, it should be feasible to determine two what other part of the sentence the role connects to.

\* In this step, token refers to the "tokenization" as applied to the PMB, i.e. the tokens in the "en.tok.off" files. E.g., "Alfred Nobel" is one token here.
Our LLM will tokenize our sentence differently, and will create one or more tokens per PMB token. This mapping will be handled later.

In [ ]:
# ROLES/LABELS: Agent, Location, Topic, Patient, Theme, EMPTY
# Tags: 0=EMPTY, 1=Agent, 2=Location, 3=Patient, 4=Theme, 5=Topic

# sentence = "A brown dog and a grey dog are fighting in the snow"
# The goal is to generate:
# srl_tags = [1,1,1,1,1,1,1,0,0,2,2,2]b
# tokens = ['A', 'brown', 'dog', 'and', 'a', 'grey', 'dog', 'are', 'fighting', 'in', 'the', 'snow']

In [1]:
import re
import os
# Example with one sentence:
# Note: forward slashes for Linux and WSL, backward slashes for Windows
# Windows example:
# file_path = r'C:\Users\bikow\Documents\AI\MSc\Computational Semantics\pmb-sample-4.0.0\data\en\gold\p00\d0004'
# WSL example:
file_path = r'/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0004/'
# file_path = r'/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p01/d2590/' # https://pmb.let.rug.nl/explorer/explore.php?part=01&doc_id=2590&type=der.xml&alignment_language=en
# file_path = r'/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p03/d0766/' # https://pmb.let.rug.nl/explorer/explore.php?part=03&doc_id=0766&type=der.xml&alignment_language=en

In [2]:
# THIS IS THE GOAL
# sentence = "A brown dog and a grey dog are fighting in the snow"
mapping = {"Agent": 1, "Location": 2, "Patient": 3, "Theme": 4, "Topic":5, "Destination": 6, "Result": 7}

sentence = ""
sentence_id = '0'
tokens = []

# Get the tokens from the tokenized sentence file
with open(file_path+"en.tok.off") as file:
    for line in file:
        tokens.append(line.split(maxsplit = 3)[-1].rstrip())

sentence = ' '.join(tokens)

print(sentence)
print(tokens)

A brown dog and a grey dog are fighting in the snow
['A', 'brown', 'dog', 'and', 'a', 'grey', 'dog', 'are', 'fighting', 'in', 'the', 'snow']


## Our class-based approach
We take the en.parse.tags file and recreate the CCG structure using custom classes.
This allows us to figure out to what tokens each semantic role label belongs.

In [16]:
class CCGNode:
    def __init__(self, category = 'none', rule_type='none', parent=None, level = 0):
        self.category = category # eg s\np or np
        self.rule_type = rule_type # fa or ba or conj
        self.children = []
        self.parent = parent
        self.level = level
        self.isFirstArgument = True
    
    def addChild(self, child):
        if len(self.children) == 1:
            child.isFirstArgument = False
        elif len(self.children) == 2:
            raise Exception(repr(self), 'already has two children')
        child.level = self.level + 1
        self.children.append(child)
    
    def getSibling(self):
        if self.isFirstArgument:
            return self.parent.children[1]
        else:
            return self.parent.children[0]
    
    def assignTag(self, tag, tagFromTokenIdx):
        self.children[0].assignTag(tag, tagFromTokenIdx)
        if len(self.children) > 1:
            self.children[1].assignTag(tag, tagFromTokenIdx)
    
    def getTags(self, mapping = None):
        if len(self.children) == 1:
            return self.children[0].getTags(mapping)
        return self.children[0].getTags(mapping) + self.children[1].getTags(mapping)
    
    
    def getTagsFromTokenIdx(self):
        if len(self.children) == 1:
            return self.children[0].getTagsFromTokenIdx()
        return self.children[0].getTagsFromTokenIdx() + self.children[1].getTagsFromTokenIdx()
    
    def __repr__(self):
        return ''.join([' ' * self.level, 'CCGNODE', ' ', self.category, ' ', self.rule_type, '\n', '\n'.join([repr(child) for child in self.children])])

class CCGToken:
    def __init__(self, token, category, parent, assignedTag = '', verbnet = [], tokenIdx = 0):
        self.token = token
        self.category = category
        self.parent = parent
        self.assignedTag = assignedTag
        self.verbnet = verbnet
        self.children = []
        self.level = None
        self.isFirstArgument = True
        self.tokenIdx = tokenIdx
        self.tagFromTokenIdx = None
        
    def getSibling(self):
        if self.isFirstArgument:
            return self.parent.children[1]
        else:
            return self.parent.children[0]
    
    def assignTag(self, tag, tagFromTokenIdx):
        self.assignedTag = tag
        self.tagFromTokenIdx = tagFromTokenIdx
    
    def getTags(self, mapping):
        if mapping == None:
            return [self.assignedTag]
        else:
            if self.assignedTag == '':
                return [0]
            return [mapping[self.assignedTag]]
    
    def getTagsFromTokenIdx(self):
        return [self.tagFromTokenIdx]
    
    def __repr__(self):
        return ''.join([' ' * self.level, 'CCGTOKEN', ' ', self.token, ' ', self.category, ' ', self.assignedTag, ' ',' '.join(self.verbnet)])


In [10]:
def getTokens(file_path):
    tokens = []
    # Get the tokens from the tokenized sentence file
    with open(os.path.join(file_path, "en.tok.off")) as file:
        for line in file:
            token = line.split(maxsplit = 3)[-1].rstrip()
            tokens.append(token)
            if token in ['.', '?', '!']:
                break
        
    return tokens

In [11]:
def getTree(file_path, tokens):
    tokenIdx = 0
    topNode = None
    currentNode = None
    tokensWithVerbnet = []
    with open(os.path.join(file_path, "en.parse.tags")) as file:
        skipping = True
        for line in file:
            if skipping:
                if line.startswith('ccg'):
                    skipping = False
                    topNode = CCGNode()
                    currentNode = topNode
                continue
            if line == '\n':
                continue
            if line.startswith('ccg'): # Second sentence starts, we ignore this
                return topNode, tokensWithVerbnet
            trimmedLine = line.lstrip()
            nodeType, content = trimmedLine.split('(', 1)
            category = content.split(',')[0]
            level = len(line) - len(trimmedLine)
            while level <= currentNode.level:
                currentNode = currentNode.parent
            if nodeType == 't':
                if category in ['.']:
                    continue
                
                vnSplit = content.split("verbnet:")
                if len(vnSplit) == 1:
                    verbnet = []
                else:
                    # It needs to combine to an np. Verbnet tags looking for a n for example,
                    # often describe adjectives and are not relevant for the main predicate
                    searchingFor = re.split(r'[\\\/]', category, 1)
                    if len(searchingFor) > 1 and ("np" in searchingFor[1]):
                        verbnetLiteral = vnSplit[1].split(']')[0] + ']'
                        verbnetUnfiltered = eval(verbnetLiteral)
                        # If first element gets filtered out but not the second, replace with dummy value
                        verbnet = [r if r in mapping.keys() else '' for r in verbnetUnfiltered]
                        # Remove trailing dummy values
                        while (verbnet) and (verbnet[-1] == ''):
                            verbnet.pop()
                    else:
                        verbnet = []
                currentNode.addChild(CCGToken(tokens[tokenIdx], category = category, parent = currentNode, verbnet = verbnet, tokenIdx = tokenIdx))
                if len(verbnet) > 0:
                    tokensWithVerbnet.append(currentNode.children[-1])
                tokenIdx += 1
            else:
                currentNode.addChild(CCGNode(category, nodeType, parent=currentNode, level = level))
                currentNode = currentNode.children[-1]
#             print(topNode)


    return topNode, tokensWithVerbnet

In [32]:
def findCorrectLevel(current):
    while (not current.category.endswith('np')): # first application with non-nps
        current = current.parent
    lookingForward = (current.category[-3] == '/')
    if lookingForward:
        while ((not current.isFirstArgument) or current.parent.rule_type != 'fa'):
            current = current.parent
    else:
        while (current.isFirstArgument or current.parent.rule_type != 'ba'):
            current = current.parent
    return current

def assignTags(tokensWithVerbnet):
    for currentTokenWithVerbnet in tokensWithVerbnet:
        verbnet = currentTokenWithVerbnet.verbnet
        currentTokenIdx = round(currentTokenWithVerbnet.tokenIdx + 0.0, 1)
        for verbnetItem in verbnet:
            currentTokenWithVerbnet = findCorrectLevel(currentTokenWithVerbnet)
            sibling = currentTokenWithVerbnet.getSibling()
            sibling.assignTag(verbnetItem, currentTokenIdx)
            currentTokenWithVerbnet = currentTokenWithVerbnet.parent
            currentTokenIdx = round(currentTokenIdx + 0.1, 1)

In [33]:
mapping = {"Agent": 1, "Location": 2, "Patient": 3, "Theme": 4, "Topic":5, "Destination": 6, "Result": 7}

# file_path = r'/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1382'
# file_path = r'/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p01/d2590/'
# file_path = r'/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0004/' # fighting dogs
file_path = r'/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p03/d0766/' # Nobel invented dynamite, two verbnet from one token
def getTokensAndLabels(file_path):
    tokens = getTokens(file_path)
    topNode, tokensWithVerbnet = getTree(file_path, tokens)
    try:
        assignTags(tokensWithVerbnet)
    except AttributeError:
        pass
    labels = topNode.getTags(mapping)
    origin = topNode.getTagsFromTokenIdx()
    if tokens[-1] not in ['.', '!', '?']:
        tokens.append('.')
    if len(tokens) > len(labels):
        labels.append(0)
        origin.append(0)
    if len(tokens) != len(labels):
        raise Exception(file_path, 'Length of token and labels does not match up!')
    return tokens, labels, origin
    

tokens, labels, origin = getTokensAndLabels(file_path)
print(tokens)
print(labels)
print(origin)

['Alfred Nobel', 'invented', 'dynamite', 'in', '1866', '.']
[1, 0, 7, 0, 0, 0]
[1.1, None, 1.0, None, None, 0]


In [ ]:
folder_path = r'/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00' 

def createDataset(parent_dir):
    dataset = {'tokens': [], 'labels': [], 'origin': []}
    for subdir, dirs, files in os.walk(parent_dir):
        if not os.path.exists(os.path.join(subdir, 'en.parse.tags')):
            continue
        print(subdir)
        tokens, labels, origin = getTokensAndLabels(subdir)
        dataset['tokens'].append(tokens)
        dataset['labels'].append(labels)
        dataset['origin'].append(origin)
    return dataset
        

dataset = createDataset(folder_path)


/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0004
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0028
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0054
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0055
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0123
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0182
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0240
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0712
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0715
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0716
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d0717
/mnt/c/Users/perry/Do

/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1466
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1468
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1469
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1470
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1472
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1474
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1480
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1487
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1489
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1491
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1493
/mnt/c/Users/perry/Do

/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1949
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1957
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1962
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1965
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1968
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1971
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1974
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1976
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1988
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1994
/mnt/c/Users/perry/Documents/uni/Master/CompSem/project/pmb-4.0.0/data/en/gold/p00/d1998
/mnt/c/Users/perry/Do

In [27]:
print(dataset)

{'tokens': [['A', 'brown', 'dog', 'and', 'a', 'grey', 'dog', 'are', 'fighting', 'in', 'the', 'snow', '.'], ['A', 'woman', 'in', 'a', 'white', 'dress', 'and', 'a', 'woman', 'in', 'a', 'blue', 'dress', 'are', 'standing', 'on', 'a', 'stage', '.'], ['There', 'is', 'no', 'man', 'playing', 'two', 'keyboards', '.'], ['There', 'is', 'no', 'person', 'cutting', 'some', 'ginger', '.'], ['You', 'are', 'screwed', '.'], ['This', 'is', 'a', 'man-made', 'language', '.'], ['which', 'singer', 'do', 'you', 'like', '?'], ['Kraft', 'sold', 'Celestial Seasonings', '.'], ['Anna Politkovskaya', 'was', 'murdered', '.'], ['Alfred Nobel', 'is', 'the', 'inventor', 'of', 'dynamite', '.'], ['The', 'yakuza', 'are', 'the', 'Japanese', 'mafia', '.'], ['Russia', 'fears', 'the', 'system', '.'], ['Bountiful', 'reached', 'San Francisco', 'on', '1', 'November', '1945', '.'], ['Pierce', 'lives', 'near', 'Rossville Blvd', '.'], ['Yunus', 'founded', 'the', 'Grameen Bank', '30', 'years', 'ago', '.'], ['Maria', 'has', 'long', '

In [46]:
from datasets import Dataset

In [47]:
ds = Dataset.from_dict(dataset)


In [48]:
ds[0]

{'tokens': ['A',
  'brown',
  'dog',
  'and',
  'a',
  'grey',
  'dog',
  'are',
  'fighting',
  'in',
  'the',
  'snow',
  '.'],
 'labels': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 2, 0]}

In [50]:
ds.save_to_disk("test.hf")

Saving the dataset (0/1 shards):   0%|          | 0/423 [00:00<?, ? examples/s]

In [56]:
from datasets import load_from_disk

In [57]:
test_dataset = load_from_disk("test.hf")

In [58]:
test_dataset

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 423
})